In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score, f1_score, precision_recall_curve)
from scapy.all import PcapReader, IP, TCP, UDP, ICMP, DNS
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from datetime import datetime
from tqdm import tqdm
import pickle
import os
import warnings
warnings.filterwarnings('ignore')


class EnsembleFeatureExtractor:
    def __init__(self, sequence_length=50):
        self.sequence_length = sequence_length
        
    def extract_packet_features(self, packet):
        features = []
        
        try:
            if IP in packet:
                features.append(min(len(packet) / 1500.0, 1.0))
                features.append(packet[IP].ttl / 255.0)
                features.append(int(packet[IP].flags) / 7.0)
                features.append((packet[IP].id % 1000) / 1000.0)
                
                if TCP in packet:
                    features.append(packet[TCP].sport / 65535.0)
                    features.append(packet[TCP].dport / 65535.0)
                    features.append(int(packet[TCP].flags) / 63.0)
                    features.append(packet[TCP].window / 65535.0)
                    payload_len = len(packet[TCP].payload)
                    features.append(min(payload_len / 1500.0, 1.0))
                    features.append(min(float(len(packet[TCP].options)) / 10.0, 1.0))
                    features.append(self._payload_entropy(bytes(packet[TCP].payload)))
                    features.extend([0.0, 0.0])
                    
                elif UDP in packet:
                    features.append(packet[UDP].sport / 65535.0)
                    features.append(packet[UDP].dport / 65535.0)
                    payload_len = len(packet[UDP].payload)
                    features.append(min(payload_len / 1500.0, 1.0))
                    features.extend([0.0, 0.0, 0.0, 0.0, 0.0])
                    
                else:
                    features.extend([0.0] * 10)
                
                features.append(1.0 if ICMP in packet else 0.0)
                features.append(1.0 if DNS in packet else 0.0)
            else:
                features = [0.0] * 13
        except Exception as e:
            # Debug: Print what failed
            # print(f"   Feature extraction error: {e}")
            features = [0.0] * 13
        
        # Ensure exactly 13 features
        if len(features) < 13:
            features.extend([0.0] * (13 - len(features)))
        
        return np.array(features[:13])
    
    def _payload_entropy(self, payload):
        """Calculate normalized Shannon entropy"""
        if not payload or len(payload) == 0:
            return 0.0
        
        byte_counts = np.bincount(list(payload), minlength=256)
        probabilities = byte_counts[byte_counts > 0] / len(payload)
        ent = entropy(probabilities, base=2) / 8.0
        return min(ent, 1.0)
    
    def extract_flow_statistical_features(self, packets_array):
        """Extract 65 statistical features from packet sequence"""
        stat_features = []
        
        # Ensure packets_array has the right shape
        if len(packets_array.shape) != 2 or packets_array.shape[1] != 13:
            # Fallback: return zero features
            return np.zeros(65)
        
        for feat_idx in range(13):
            feat_values = packets_array[:, feat_idx]
            
            # Handle edge cases
            if len(feat_values) == 0:
                stat_features.extend([0.0, 0.0, 0.0, 0.0, 0.0])
            else:
                stat_features.extend([
                    np.mean(feat_values),
                    np.std(feat_values) if len(feat_values) > 1 else 0.0,
                    np.min(feat_values),
                    np.max(feat_values),
                    np.median(feat_values)
                ])
        
        return np.array(stat_features)  # 65 features
    
    def process_pcap_for_ensemble(self, pcap_file, label, max_flows=500):
        """Extract both sequence and statistical features"""
        print(f"\nProcessing {pcap_file}...")
        
        flow_dict = defaultdict(list)
        
        try:
            with PcapReader(pcap_file) as pcap:
                for packet in tqdm(pcap, desc="   Reading packets"):
                    if IP not in packet:
                        continue
                    
                    src_ip = packet[IP].src
                    dst_ip = packet[IP].dst
                    proto = packet[IP].proto
                    
                    if TCP in packet:
                        src_port = packet[TCP].sport
                        dst_port = packet[TCP].dport
                    elif UDP in packet:
                        src_port = packet[UDP].sport
                        dst_port = packet[UDP].dport
                    else:
                        src_port = 0
                        dst_port = 0
                    
                    flow_key = tuple(sorted([
                        (src_ip, src_port, proto),
                        (dst_ip, dst_port, proto)
                    ]))
                    
                    features = self.extract_packet_features(packet)
                    flow_dict[flow_key].append(features)
            
            print(f"   Extracted {len(flow_dict)} flows")
            
            # Create both sequence and statistical features
            sequences = []
            stat_features = []
            labels = []
            
            # Debug: Check flow sizes
            flow_sizes = [len(packets) for packets in flow_dict.values()]
            if flow_sizes:
                print(f"   Flow size stats: min={min(flow_sizes)}, max={max(flow_sizes)}, "
                      f"avg={np.mean(flow_sizes):.1f}, median={np.median(flow_sizes):.0f}")
                flows_with_3plus = sum(1 for s in flow_sizes if s >= 3)
                print(f"   Flows with ≥3 packets: {flows_with_3plus}/{len(flow_sizes)} ({flows_with_3plus/len(flow_sizes)*100:.1f}%)")
            
            # REDUCED THRESHOLD: Use flows with at least 1 packet (will pad to sequence_length)
            min_packets = 1  # Changed from 3 to 1
            
            for flow_key, packets in tqdm(list(flow_dict.items())[:max_flows], desc="   Creating features"):
                if len(packets) >= min_packets:
                    # Pad/truncate for sequence
                    if len(packets) < self.sequence_length:
                        packets = packets + [np.zeros(13)] * (self.sequence_length - len(packets))
                    else:
                        packets = packets[:self.sequence_length]
                    
                    packets_array = np.array(packets)
                    sequences.append(packets_array)
                    stat_features.append(self.extract_flow_statistical_features(packets_array))
                    labels.append(label)
            
            # Ensure we return proper shapes even if empty
            if len(sequences) == 0:
                print(f"    Warning: No valid flows found (tried {len(flow_dict)} flows)")
                return (np.empty((0, self.sequence_length, 13)), 
                       np.empty((0, 65)), 
                       np.empty((0,)))
            
            print(f"    Created {len(sequences)} valid sequences from {min(max_flows, len(flow_dict))} flows")
            return np.array(sequences), np.array(stat_features), np.array(labels)
            
        except Exception as e:
            print(f"    Error: {e}")
            import traceback
            traceback.print_exc()
            return (np.empty((0, self.sequence_length, 13)), 
                   np.empty((0, 65)), 
                   np.empty((0,)))


class EnsembleSteganographyDetector:
    """Ensemble model combining BiLSTM + ML classifiers"""
    
    def __init__(self, sequence_length=50, n_features=13):
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.bilstm_model = None
        self.rf_model = None
        self.gb_model = None
        self.lr_model = None
        self.voting_model = None
        self.scaler_seq = StandardScaler()
        self.scaler_stat = StandardScaler()
        self.history = None
        
    def build_bilstm_model(self):
        """Build BiLSTM model"""
        print("\n" + "="*70)
        print(" BUILDING BiLSTM MODEL")
        print("="*70)
        
        self.bilstm_model = models.Sequential([
            layers.Input(shape=(self.sequence_length, self.n_features)),
            layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.2)),
            layers.Dropout(0.3),
            layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.2)),
            layers.Dropout(0.2),
            layers.Bidirectional(layers.LSTM(32, return_sequences=False, dropout=0.2)),
            layers.Dropout(0.2),
            layers.Dense(64, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(32, activation='relu'),
            layers.Dropout(0.1),
            layers.Dense(1, activation='sigmoid')
        ])
        
        self.bilstm_model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', keras.metrics.AUC(name='auc')]
        )
        
        print(" BiLSTM model built")
    
    def build_ml_models(self):
        print("\n" + "="*70)
        print(" BUILDING ML MODELS")
        print("="*70)
        
        self.rf_model = RandomForestClassifier(
            n_estimators=200, 
            max_depth=20,
            min_samples_split=5,
            random_state=42,
            n_jobs=-1
        )
        
        self.gb_model = GradientBoostingClassifier(
            n_estimators=100,
            max_depth=10,
            learning_rate=0.1,
            random_state=42
        )
        
        self.lr_model = LogisticRegression(
            max_iter=1000,
            random_state=42
        )
        
        print(" Random Forest model initialized")
        print(" Gradient Boosting model initialized")
        print(" Logistic Regression model initialized")
    
    def train_ensemble(self, X_seq_train, X_stat_train, y_train,
                      X_seq_val, X_stat_val, y_val,
                      epochs=50, batch_size=32):
        """Train all models"""
        print("\n" + "="*70)
        print(" TRAINING ENSEMBLE")
        print("="*70)
        
        # Normalize sequence features
        print("\nNormalizing sequence features...")
        seq_shape = X_seq_train.shape
        X_seq_train_flat = X_seq_train.reshape(-1, self.n_features)
        X_seq_val_flat = X_seq_val.reshape(-1, self.n_features)
        
        X_seq_train_scaled = self.scaler_seq.fit_transform(X_seq_train_flat)
        X_seq_val_scaled = self.scaler_seq.transform(X_seq_val_flat)
        
        X_seq_train = X_seq_train_scaled.reshape(seq_shape)
        X_seq_val = X_seq_val_scaled.reshape(X_seq_val.shape)
        
        # Normalize statistical features
        print("Normalizing statistical features...")
        X_stat_train = self.scaler_stat.fit_transform(X_stat_train)
        X_stat_val = self.scaler_stat.transform(X_stat_val)
        
        # Train BiLSTM
        print("\n[1/4] Training BiLSTM...")
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=8,
                min_lr=1e-6,
                verbose=1
            )
        ]
        
        self.history = self.bilstm_model.fit(
            X_seq_train, y_train,
            validation_data=(X_seq_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )
        print(" BiLSTM training complete")
        
        # Train Random Forest
        print("\n[2/4] Training Random Forest...")
        self.rf_model.fit(X_stat_train, y_train)
        rf_score = self.rf_model.score(X_stat_val, y_val)
        print(f" Random Forest validation accuracy: {rf_score:.4f}")
        
        # Train Gradient Boosting
        print("\n[3/4] Training Gradient Boosting...")
        self.gb_model.fit(X_stat_train, y_train)
        gb_score = self.gb_model.score(X_stat_val, y_val)
        print(f" Gradient Boosting validation accuracy: {gb_score:.4f}")
        
        # Train Logistic Regression
        print("\n[4/4] Training Logistic Regression...")
        self.lr_model.fit(X_stat_train, y_train)
        lr_score = self.lr_model.score(X_stat_val, y_val)
        print(f" Logistic Regression validation accuracy: {lr_score:.4f}")
        
        return X_seq_train, X_stat_train, X_seq_val, X_stat_val
    
    def predict_ensemble(self, X_seq, X_stat, method='weighted_average'):
        """
        Ensemble prediction methods:
        - 'weighted_average': Weighted average of all predictions
        - 'majority_vote': Majority voting
        - 'stacking': Use predictions as features
        """
        # Get individual predictions
        bilstm_pred = self.bilstm_model.predict(X_seq, verbose=0).flatten()
        rf_pred = self.rf_model.predict_proba(X_stat)[:, 1]
        gb_pred = self.gb_model.predict_proba(X_stat)[:, 1]
        lr_pred = self.lr_model.predict_proba(X_stat)[:, 1]
        
        if method == 'weighted_average':
            # Weighted average (BiLSTM has highest weight)
            weights = [0.4, 0.25, 0.25, 0.1]  # BiLSTM, RF, GB, LR
            ensemble_pred = (
                weights[0] * bilstm_pred +
                weights[1] * rf_pred +
                weights[2] * gb_pred +
                weights[3] * lr_pred
            )
            
        elif method == 'majority_vote':
            # Convert to binary and vote
            bilstm_binary = (bilstm_pred > 0.5).astype(int)
            rf_binary = (rf_pred > 0.5).astype(int)
            gb_binary = (gb_pred > 0.5).astype(int)
            lr_binary = (lr_pred > 0.5).astype(int)
            
            votes = bilstm_binary + rf_binary + gb_binary + lr_binary
            ensemble_pred = (votes >= 2).astype(float)
            
        else:  # max
            ensemble_pred = np.maximum(np.maximum(bilstm_pred, rf_pred), 
                                      np.maximum(gb_pred, lr_pred))
        
        return ensemble_pred, {
            'bilstm': bilstm_pred,
            'rf': rf_pred,
            'gb': gb_pred,
            'lr': lr_pred
        }
    
    def evaluate_ensemble(self, X_seq_test, X_stat_test, y_test, method='weighted_average'):
        """Evaluate ensemble performance"""
        print("\n" + "="*70)
        print(" ENSEMBLE EVALUATION")
        print("="*70)
        
        # Normalize test data
        X_seq_test_flat = X_seq_test.reshape(-1, self.n_features)
        X_seq_test_scaled = self.scaler_seq.transform(X_seq_test_flat)
        X_seq_test = X_seq_test_scaled.reshape(X_seq_test.shape)
        X_stat_test = self.scaler_stat.transform(X_stat_test)
        
        # Get ensemble predictions
        ensemble_pred_proba, individual_preds = self.predict_ensemble(
            X_seq_test, X_stat_test, method=method
        )
        ensemble_pred = (ensemble_pred_proba > 0.5).astype(int)
        
        # Metrics
        accuracy = accuracy_score(y_test, ensemble_pred)
        f1 = f1_score(y_test, ensemble_pred)
        auc_score = roc_auc_score(y_test, ensemble_pred_proba)
        
        print(f"\n Ensemble Performance ({method}):")
        print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"   F1-Score:  {f1:.4f}")
        print(f"   AUC-ROC:   {auc_score:.4f}")
        
        # Individual model performance
        print(f"\n Individual Model Performance:")
        for model_name, pred_proba in individual_preds.items():
            pred = (pred_proba > 0.5).astype(int)
            acc = accuracy_score(y_test, pred)
            f1_ind = f1_score(y_test, pred)
            print(f"   {model_name.upper():<10} Acc: {acc:.4f}  F1: {f1_ind:.4f}")
        
        print(f"\n Classification Report:")
        print(classification_report(y_test, ensemble_pred, 
                                   target_names=['Benign', 'Steganography']))
        
        cm = confusion_matrix(y_test, ensemble_pred)
        print(f"\n Confusion Matrix:")
        print(f"   True Negatives:  {cm[0,0]:>6,}")
        print(f"   False Positives: {cm[0,1]:>6,}")
        print(f"   False Negatives: {cm[1,0]:>6,}")
        print(f"   True Positives:  {cm[1,1]:>6,}")
        
        return ensemble_pred, ensemble_pred_proba, individual_preds, cm
    
    def plot_results(self, y_test, ensemble_pred, ensemble_pred_proba, 
                    individual_preds, cm, output_dir='results'):
        """Plot comprehensive results"""
        print("\n Generating visualizations...")
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        fig = plt.figure(figsize=(18, 12))
        
        # 1. Confusion Matrix
        ax1 = plt.subplot(2, 3, 1)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, ax=ax1,
                   xticklabels=['Benign', 'Stego'],
                   yticklabels=['Benign', 'Stego'])
        ax1.set_title('Ensemble Confusion Matrix', fontsize=12, fontweight='bold')
        
        # 2. ROC Curves
        ax2 = plt.subplot(2, 3, 2)
        
        # Ensemble ROC
        fpr_ens, tpr_ens, _ = roc_curve(y_test, ensemble_pred_proba)
        auc_ens = roc_auc_score(y_test, ensemble_pred_proba)
        ax2.plot(fpr_ens, tpr_ens, label=f'Ensemble (AUC={auc_ens:.4f})', 
                linewidth=3, color='red')
        
        # Individual ROCs
        colors = ['blue', 'green', 'orange', 'purple']
        for (name, pred), color in zip(individual_preds.items(), colors):
            fpr, tpr, _ = roc_curve(y_test, pred)
            auc_score = roc_auc_score(y_test, pred)
            ax2.plot(fpr, tpr, label=f'{name.upper()} (AUC={auc_score:.4f})',
                    linewidth=2, alpha=0.7, color=color)
        
        ax2.plot([0, 1], [0, 1], 'k--', linewidth=1)
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.set_title('ROC Curves Comparison', fontsize=12, fontweight='bold')
        ax2.legend(fontsize=8)
        ax2.grid(True, alpha=0.3)
        
        # 3. Model Accuracy Comparison
        ax3 = plt.subplot(2, 3, 3)
        model_names = ['BiLSTM', 'RF', 'GB', 'LR', 'Ensemble']
        accuracies = []
        for name, pred in individual_preds.items():
            accuracies.append(accuracy_score(y_test, (pred > 0.5).astype(int)))
        accuracies.append(accuracy_score(y_test, ensemble_pred))
        
        bars = ax3.bar(model_names, accuracies, 
                      color=['#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#e74c3c'])
        ax3.set_ylabel('Accuracy')
        ax3.set_title('Model Accuracy Comparison', fontsize=12, fontweight='bold')
        ax3.set_ylim([0.9, 1.0])
        ax3.grid(True, alpha=0.3, axis='y')
        
        for bar, acc in zip(bars, accuracies):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')
        
        # 4. Prediction Distribution
        ax4 = plt.subplot(2, 3, 4)
        ax4.hist(ensemble_pred_proba[y_test == 0], bins=30, alpha=0.7, 
                label='Benign', color='green', edgecolor='black')
        ax4.hist(ensemble_pred_proba[y_test == 1], bins=30, alpha=0.7, 
                label='Stego', color='red', edgecolor='black')
        ax4.axvline(0.5, color='black', linestyle='--', linewidth=2, label='Threshold')
        ax4.set_xlabel('Prediction Probability')
        ax4.set_ylabel('Frequency')
        ax4.set_title('Ensemble Prediction Distribution', fontsize=12, fontweight='bold')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # 5. BiLSTM Training History
        if self.history:
            ax5 = plt.subplot(2, 3, 5)
            ax5.plot(self.history.history['accuracy'], label='Train', linewidth=2)
            ax5.plot(self.history.history['val_accuracy'], label='Val', linewidth=2)
            ax5.set_xlabel('Epoch')
            ax5.set_ylabel('Accuracy')
            ax5.set_title('BiLSTM Training History', fontsize=12, fontweight='bold')
            ax5.legend()
            ax5.grid(True, alpha=0.3)
        
        # 6. F1-Score Comparison
        ax6 = plt.subplot(2, 3, 6)
        f1_scores = []
        for name, pred in individual_preds.items():
            f1_scores.append(f1_score(y_test, (pred > 0.5).astype(int)))
        f1_scores.append(f1_score(y_test, ensemble_pred))
        
        bars = ax6.bar(model_names, f1_scores,
                      color=['#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#e74c3c'])
        ax6.set_ylabel('F1-Score')
        ax6.set_title('Model F1-Score Comparison', fontsize=12, fontweight='bold')
        ax6.set_ylim([0.9, 1.0])
        ax6.grid(True, alpha=0.3, axis='y')
        
        for bar, f1 in zip(bars, f1_scores):
            height = bar.get_height()
            ax6.text(bar.get_x() + bar.get_width()/2., height,
                    f'{f1:.4f}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        
        plot_path = os.path.join(output_dir, 'ensemble_results.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f" Saved: {plot_path}")
        plt.close()
    
    def save_models(self, output_dir='models'):
        """Save all trained models"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print("\n" + "="*70)
        print(" SAVING ENSEMBLE MODELS")
        print("="*70)
        
        # Save BiLSTM
        bilstm_path = os.path.join(output_dir, f'ensemble_bilstm_{timestamp}.h5')
        self.bilstm_model.save(bilstm_path)
        print(f"\n✓ BiLSTM saved: {bilstm_path}")
        
        # Save ML models
        ensemble_dict = {
            'rf': self.rf_model,
            'gb': self.gb_model,
            'lr': self.lr_model,
            'scaler_seq': self.scaler_seq,
            'scaler_stat': self.scaler_stat,
            'metadata': {
                'timestamp': timestamp,
                'sequence_length': self.sequence_length,
                'n_features': self.n_features
            }
        }
        
        ensemble_path = os.path.join(output_dir, f'ensemble_models_{timestamp}.pkl')
        with open(ensemble_path, 'wb') as f:
            pickle.dump(ensemble_dict, f)
        print(f" ML models saved: {ensemble_path}")
        
        print(f"\n All models saved in {output_dir}/")


def main():
    """Main ensemble training pipeline"""
    
    benign_pcap = "Dataset/Benign_Dump.pcap"
    stego_pcap = "Dataset/steganography_dataset_20251016_233034.pcap"
    
    # Check if files exist
    if not os.path.exists(benign_pcap):
        print(f" Benign PCAP not found: {benign_pcap}")
        return
    if not os.path.exists(stego_pcap):
        print(f" Stego PCAP not found: {stego_pcap}")
        return
    
    # Extract features
    print("\n" + "="*70)
    print(" EXTRACTING FEATURES FROM PCAPS")
    print("="*70)
    
    extractor = EnsembleFeatureExtractor(sequence_length=50)
    
    # Process both PCAPs
    X_seq_benign, X_stat_benign, y_benign = extractor.process_pcap_for_ensemble(
        benign_pcap, label=0, max_flows=500
    )
    
    X_seq_stego, X_stat_stego, y_stego = extractor.process_pcap_for_ensemble(
        stego_pcap, label=1, max_flows=500
    )
    
    # Validate data
    if len(X_seq_benign) == 0 or len(X_seq_stego) == 0:
        print("\n Error: One or both datasets are empty!")
        print(f"   Benign flows: {len(X_seq_benign)}")
        print(f"   Stego flows: {len(X_seq_stego)}")
        return
    
    print(f"\n Feature Extraction Complete:")
    print(f"   Benign: {len(X_seq_benign)} flows")
    print(f"   Stego:  {len(X_seq_stego)} flows")
    
    # Combine datasets
    X_seq = np.concatenate([X_seq_benign, X_seq_stego], axis=0)
    X_stat = np.concatenate([X_stat_benign, X_stat_stego], axis=0)
    y = np.concatenate([y_benign, y_stego], axis=0)
    
    print(f"\n Combined Dataset:")
    print(f"   Sequences shape: {X_seq.shape}")
    print(f"   Statistical shape: {X_stat.shape}")
    print(f"   Labels shape: {y.shape}")
    print(f"   Total samples: {len(y)}")
    
    # Check minimum samples
    if len(y) < 100:
        print(f"\n Warning: Only {len(y)} samples. Need at least 100 for reliable training.")
        print("   Consider increasing max_flows or using more PCAP files.")
        return
    
    # Split data
    print(f"\n Splitting data...")
    indices = np.arange(len(y))
    train_idx, test_idx = train_test_split(indices, test_size=0.2, 
                                          stratify=y, random_state=42)
    train_idx, val_idx = train_test_split(train_idx, test_size=0.2,
                                         stratify=y[train_idx], random_state=42)
    
    X_seq_train, X_seq_val, X_seq_test = X_seq[train_idx], X_seq[val_idx], X_seq[test_idx]
    X_stat_train, X_stat_val, X_stat_test = X_stat[train_idx], X_stat[val_idx], X_stat[test_idx]
    y_train, y_val, y_test = y[train_idx], y[val_idx], y[test_idx]
    
    print(f" Data Split:")
    print(f"   Train: {len(train_idx)} samples")
    print(f"   Val:   {len(val_idx)} samples")
    print(f"   Test:  {len(test_idx)} samples")
    
    # Initialize ensemble
    ensemble = EnsembleSteganographyDetector(sequence_length=50, n_features=13)
    
    # Build models
    ensemble.build_bilstm_model()
    ensemble.build_ml_models()
    
    # Train ensemble
    try:
        X_seq_train, X_stat_train, X_seq_val, X_stat_val = ensemble.train_ensemble(
            X_seq_train, X_stat_train, y_train,
            X_seq_val, X_stat_val, y_val,
            epochs=10, batch_size=32
        )
    except Exception as e:
        print(f"\n Training failed: {e}")
        import traceback
        traceback.print_exc()
        return
    
    # Evaluate
    try:
        ensemble_pred, ensemble_pred_proba, individual_preds, cm = ensemble.evaluate_ensemble(
            X_seq_test, X_stat_test, y_test, method='weighted_average'
        )
    except Exception as e:
        print(f"\n Evaluation failed: {e}")
        import traceback
        traceback.print_exc()
        return
    
    # Plot results
    try:
        ensemble.plot_results(y_test, ensemble_pred, ensemble_pred_proba,
                             individual_preds, cm)
    except Exception as e:
        print(f"\n Plotting failed: {e}")
        print("   Continuing without plots...")
    
    # Save models
    try:
        ensemble.save_models()
    except Exception as e:
        print(f"\n Model saving failed: {e}")
        print("   Models were trained but not saved.")
    
    print("\n" + "="*70)
    print("  ENSEMBLE TRAINING COMPLETE!")
    print("="*70)
    print("\n Check ./results/ for visualizations")
    print(" Check ./models/ for saved models\n")


if __name__ == "__main__":
    main()


 EXTRACTING FEATURES FROM PCAPS

Processing Dataset/Benign_Dump.pcap...


   Reading packets: 0it [00:00, ?it/s]

   Reading packets: 86000it [00:21, 4083.67it/s]


   ✓ Extracted 38444 flows
   Flow size stats: min=1, max=15853, avg=2.2, median=1
   Flows with ≥3 packets: 611/38444 (1.6%)


   Creating features: 100%|██████████| 500/500 [00:00<00:00, 1402.34it/s]


...

   Reading packets: 92067it [00:35, 2585.85it/s]


   ✓ Extracted 81985 flows
   Flow size stats: min=1, max=7, avg=1.1, median=1
   Flows with ≥3 packets: 1062/81985 (1.3%)


   Creating features: 100%|██████████| 500/500 [00:00<00:00, 1393.34it/s]


   ✓ Created 500 valid sequences from 500 flows

✓ Feature Extraction Complete:
   Benign: 500 flows
   Stego:  500 flows

✓ Combined Dataset:
   Sequences shape: (1000, 50, 13)
   Statistical shape: (1000, 65)
   Labels shape: (1000,)
   Total samples: 1000

🔀 Splitting data...
✓ Data Split:
   Train: 640 samples
   Val:   160 samples
   Test:  200 samples

 BUILDING BiLSTM MODEL
✓ BiLSTM model built

 BUILDING ML MODELS
✓ Random Forest model initialized
✓ Gradient Boosting model initialized
✓ Logistic Regression model initialized

 TRAINING ENSEMBLE

Normalizing sequence features...
Normalizing statistical features...

[1/4] Training BiLSTM...
Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1:58 6s/step - accuracy: 0.4375 - auc: 0.4039 - loss: 0.843 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.4688 - auc: 0.4436 - loss: 0.835 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.5139 - auc: 0.5042 - loss: 0.799 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.5514 - auc: 0.5573 - loss: 0.763 ━━━━━


✓ BiLSTM saved: models\ensemble_bilstm_20251107_004524.h5
✓ ML models saved: models\ensemble_models_20251107_004524.pkl

✓ All models saved in models/

 ✅ ENSEMBLE TRAINING COMPLETE!

📁 Check ./results/ for visualizations
📁 Check ./models/ for saved models

